In [1]:
!git clone https://github.com/vigneshbabupj/Project_Vision.git

Cloning into 'Project_Vision'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1931 (delta 30), reused 39 (delta 15), pack-reused 1877
Receiving objects: 100% (1931/1931), 1.24 MiB | 28.93 MiB/s, done.
Resolving deltas: 100% (1297/1297), done.


In [2]:
!pip install git+https://github.com/longcw/RoIAlign.pytorch -q

In [3]:
ls

Project_Vision/  sample_data/


In [4]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [5]:
%%capture
%cd Project_Vision/
!mkdir midas
%cd midas
!wget https://github.com/intel-isl/MiDaS/releases/download/v2_1/model-f6b98070.pt
%cd ..

In [6]:
!cp '/content/gdrive/My Drive/EVA/EVA5/15A/planercnn_model/checkpoint.zip' '.'
!unzip checkpoint.zip
!ls 
!rm -r checkpoint.zip

Archive:  checkpoint.zip
   creating: planercnn_normal_warping_refine/
  inflating: planercnn_normal_warping_refine/checkpoint_refine.pth  
  inflating: planercnn_normal_warping_refine/checkpoint.pth  
anchors		detect.py   model.py			     pytorch_ssim
bbox_decoder	documents   options.py			     README.md
checkpoint.zip	encoder.py  plane_decoder		     train.py
dataset.py	LICENSE     planercnn_normal_warping_refine
depth_decoder	midas	    pytorch_msssim


In [7]:
%%capture
%cd Project_Vision/
!cp '/content/gdrive/My Drive/EVA/updated_final_data.zip' '.'

!unzip updated_final_data.zip
!rm -r updated_final_data.zip

In [8]:
ls data/customdata/

custom.data   depth_images/  labels/                    test.txt
custom.names  images/        planercnn_data_actual.zip  train.txt


In [9]:
%%capture
%cd data/customdata/
!unzip planercnn_data_actual.zip
!rm -r planercnn_data_actual.zip

In [10]:
ls 

content/     custom.names   images/  test.txt
custom.data  depth_images/  labels/  train.txt


In [11]:
!mv -v 'content/planercnn/test/inference' '.'
!cp '/content/gdrive/My Drive/EVA/EVA5/15A/custom.shapes' '.'
!cp '/content/gdrive/My Drive/EVA/EVA5/15A/camera.txt' './images'
%cd ../../
!ls

renamed 'content/planercnn/test/inference' -> './inference'
/content/Project_Vision
anchors        detect.py   model.py			    pytorch_ssim
bbox_decoder   documents   options.py			    README.md
data	       encoder.py  plane_decoder		    train.py
dataset.py     LICENSE	   planercnn_normal_warping_refine
depth_decoder  midas	   pytorch_msssim


#Run Code

In [1]:
%cd Project_Vision/
!sed -i '/-With-A-Puffy-Jacket-Boots-And-A-Belt.jpg/d' data/customdata/train.txt
#!sed -i '1,10!d' data/customdata/train.txt 
#!sed -i '1,10!d' data/customdata/test.txt
!git pull -f

/content/Project_Vision
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/vigneshbabupj/Project_Vision
   df71137..7d422c7  main       -> origin/main
Updating df71137..7d422c7
Fast-forward
 bbox_decoder/Actual_layers_sizes | 116 +++++++++++++++++++++++++++++++++++++++
 dataset.py                       |   4 +-
 2 files changed, 118 insertions(+), 2 deletions(-)
 create mode 100644 bbox_decoder/Actual_layers_sizes


In [2]:
from options import *
import sys
sys.argv = ['']
plane_args = plane_parse_args()
yolo_args = yolo_parse_args()
midas_args = midas_parse_args()

yolo_args.batch_size = 1

yolo_args.cfg = 'bbox_decoder/cfg/yolov3-custom.cfg'
yolo_args.epochs = 10
yolo_args.weights = '/content/gdrive/My Drive/EVA/EVA5/yolov3-spp-ultralytics.pt'
yolo_args.data = 'data/customdata/custom.data'
midas_args.input = 'data/customdata/images'
#midas_args.output
midas_args.weights = 'midas/model-f6b98070.pt'

plane_args.customDataFolder = 'data/customdata/images'
plane_args.checkpoint_dir = 'planercnn_normal_warping_refine'
plane_args.suffix = 'warping_refine'

yolo_args.img_size=[512,512,512] #[640,640,640]
#yolo_args.img_size=[256,256,256] #[640,640,640]
plane_args.height = 512
plane_args.width = 512
plane_args.test_dir = 'test'

In [3]:
%matplotlib inline
from train import train
loss = train(plane_args,yolo_args,midas_args,add_plane_loss=1,add_yolo_loss=1,add_midas_loss=1,resume_train=True,model_path='/content/gdrive/My Drive/EVA/EVA5/capstone/')

Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

use_pretrained True
path midas/model-f6b98070.pt


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


Model Summary: 1056 layers, 2.37595e+08 parameters, 2.37484e+08 gradients


Caching labels (3160 found, 4 missing, 38 empty, 0 duplicate, for 3202 images): 100%|██████████| 3202/3202 [00:02<00:00, 1486.10it/s]
Caching images (1.8GB): 100%|██████████| 3202/3202 [00:22<00:00, 143.34it/s]
Caching labels (311 found, 0 missing, 7 empty, 0 duplicate, for 318 images): 100%|██████████| 318/318 [00:00<00:00, 2342.20it/s]
Caching images (0.1GB): 100%|██████████| 318/318 [00:03<00:00, 103.16it/s]
0it [00:00, ?it/s]

Image sizes 512 - 512 train, 512 test
Using 0 dataloader workers
Starting training for 10 epochs...

     Epoch   Dp_loss  bbx_loss  pln_loss  All_loss  img_size


...


     Epoch   Dp_loss  bbx_loss  pln_loss  All_loss  img_size


     28/35      5.22      3.34      1.06      9.62       512: : 3202it [57:43,  1.08s/it]
0it [00:00, ?it/s]


     Epoch   Dp_loss  bbx_loss  pln_loss  All_loss  img_size


     29/35      5.34      1.54      1.08      7.96       512: : 638it [11:38,  1.07s/it]

KeyboardInterrupt: ignored

In [ ]:
%debug

In [ ]:
#from torchsummary import summary

#summary(model2, (3, 256, 256))